# HW2 - Analysis 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection._search import ParameterGrid
import seaborn as sns
import copy

In [ ]:
%matplotlib inline

In [ ]:
from whatif import Model
from whatif import get_sim_results_df

### Base Model

**This works - so there must be some reason self.demand needs () in the formula and the __init__ variables do not (in the demand and profit formulas) - 
Found online: self.demand() is called with parentheses because it's invoking a method of the class instance (self). If demand were an attribute rather than a method, you would access it without parentheses, as in self.demand.**

In [ ]:
class SingleProductSPF(Model):
    def __init__(self, fixed_cost=5000, var_cost=100, selling_price=0,
                 spf_constant=4900, spf_linear=-35, spf_quadratic=0.06):
        self.fixed_cost = fixed_cost
        self.var_cost = var_cost
        self.selling_price = selling_price
        self.spf_constant = spf_constant
        self.spf_linear = spf_linear
        self.spf_quadratic = spf_quadratic
        
    def demand(self):
        """Compute demand"""
        demand = (self.spf_quadratic * (self.selling_price ** 2)) + (self.spf_linear * self.selling_price) + self.spf_constant
        return demand
    
    def profit(self):
        """Compute profit"""
        profit = (self.demand() * self.selling_price) - (self.demand() * self.var_cost) - self.fixed_cost
        return profit
    
    def total_cost(self):
        """Computer total cost"""
        total_cost = (self.demand() * self.var_cost) - self.fixed_cost
        return total_cost


**I wanted to see if the update function would work**

In [ ]:
# Create a dictionary of base input values
base_inputs = {'selling_price': 115}

In [ ]:
# Create a new model with default input values (0's)
model_test = SingleProductSPF()
print(model_test)
model_test.demand()

In [ ]:
# Update model with base inputs
model_test.update(base_inputs)
print(model_test)

In [ ]:
model_test.demand()

In [ ]:
model_test.profit()

### Data Table Function

### 1-Way Data Table

**Originally, I had the full functions in the notebook as well, then realized they could all be referenced**

In [ ]:
# Specify input ranges for scenarios (dictionary)
inputs = {'selling_price': np.arange(80, 141, 10)}

# Specify desired outputs (list)
outputs = ['profit', 'demand']

# Use data_table function
mt_dt1_df = model_test.data_table(inputs, outputs)
mt_dt1_df

In [ ]:
# Plot
plt.plot(mt_dt1_df['selling_price'], mt_dt1_df['profit'], marker='o')

# Labeling the plot
plt.title('Selling Price vs. Profit')
plt.xlabel('Selling Price ($)')
plt.ylabel('Profit ($)')

# Show plot
plt.grid(True)
plt.show()

**It is non-linear with a curve, as the demand formula is quadratic**

https://www.cuemath.com/algebra/factorization-of-quadratic-polynomials/

A quadratic polynomial is also known as a second-order polynomial. This means that at least one of the variables should be raised to the power of two while the other variable powers will be lesser than or equal to two but greater than -1. A quadratic polynomial can have multiple variables.

### Using goal_seek

In [ ]:
model_test.goal_seek('profit', 0, 'selling_price', 80, 140, N=100)

**The breakeven selling price is $102.58**

### 2-Way Data Table

In [ ]:
# Specify input ranges for scenarios (dictionary)
inputs = {'selling_price': np.arange(80, 141, 10),
                  'var_cost': np.arange(85, 111, 5)}

# Specify desired outputs (list)
outputs = ['profit', 'total_cost']

# Use data_table function to create dataframe
mt_dt2_df = model_test.data_table(inputs, outputs)
mt_dt2_df

In [ ]:
# Scatter plot - negatives caused an error
plt.figure(figsize=(10, 6))
plt.scatter(mt_dt2_df['selling_price'], mt_dt2_df['var_cost'], s=mt_dt2_df['profit'], c=mt_dt2_df['profit'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Profit')
plt.xlabel('Selling Price')
plt.ylabel('Variable Cost')
plt.title('Profit vs Selling Price and Variable Cost')
plt.grid(True)
plt.show()


In [ ]:
# Handling NaN and negative values in 'profit' column
mt_dt2_df['profit'] = mt_dt2_df['profit'].fillna(0)  # Fill NaN with 0
mt_dt2_df['profit'] = mt_dt2_df['profit'].clip(lower=0)  # Clip negative values to 0

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(mt_dt2_df['selling_price'], mt_dt2_df['var_cost'], s=mt_dt2_df['profit'], c=mt_dt2_df['profit'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Profit')
plt.xlabel('Selling Price')
plt.ylabel('Variable Cost')
plt.title('Profit vs Selling Price and Variable Cost')
plt.grid(True)
plt.show()

**I love the visual of this plot - but I'll do something that is a little more readable**

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(mt_dt2_df['selling_price'], mt_dt2_df['var_cost'], s=100, c=mt_dt2_df['profit'], cmap='viridis', alpha=0.9)
plt.colorbar(label='Profit')
plt.xlabel('Selling Price')
plt.ylabel('Variable Cost')
plt.title('Profit vs Selling Price and Variable Cost')
plt.grid(True)
plt.show()

**While I love the 'viridis' colors, one of my group project members from my last class is colorblind and I started to think about color choices, there was supposed to be a 'colorblind' selection, but that caused an error, 'cividis' is supposed to be another good choice - I think it still depends on what colors the individual sees best**

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(mt_dt2_df['selling_price'], mt_dt2_df['var_cost'], s=100, c=mt_dt2_df['profit'], cmap='cividis', alpha=0.9)
plt.colorbar(label='Profit')
plt.xlabel('Selling Price')
plt.ylabel('Variable Cost')
plt.title('Profit vs Selling Price and Variable Cost')
plt.grid(True)
plt.show()

### Re-do goal_seek \\$80 - 250:
**I was able to get 227 to work, 228 - 250 failed - I changed N and that didn't help**  <br>
**It's the same breakeven $102.58**

In [ ]:
model_test.goal_seek('profit', 0, 'selling_price', 80, 250, N=100)

### Another 1-Way Data Table \\$80 - 250:

In [ ]:
# Specify input ranges for scenarios (dictionary)
inputs = {'selling_price': np.arange(80, 251, 10)}

# Specify desired outputs (list)
outputs = ['profit', 'demand']

# Use data_table function
mt_dt3_df = model_test.data_table(inputs, outputs)
mt_dt3_df

***LOL - oops...negatives!!! I hate it when that happens...***

### Simulation

In [ ]:
from numpy.random import default_rng
rg = default_rng(4470)
rg.random() # Generate one just to see it work

In [ ]:
#simulate(self, random_inputs, outputs, scenario_inputs=None, keep_random_inputs=False)
num_reps = 100

random_inputs = {'var_cost': rg.uniform(80.0, 120.0, num_reps)}

sim_outputs = ['profit'] 

model_test_results = model_test.simulate(random_inputs, sim_outputs)

model_test_results

In [ ]:
model_test_results_df = get_sim_results_df(model_test_results)

model_test_results_df

In [ ]:
# creating a basic histogram 

model_test_results_df.hist('profit')
plt.title("Profit histogram for random Variable Cost ranging \\$80 - \\$140")
plt.xlabel("Profit")
plt.ylabel("Num observations")
plt.show() 

***I believe this is a non-symmetric multi-modal distribution - the histogram changed when I reset the kernel to make sure everything worked***

https://www.statology.org/multimodal-distribution/

https://www.sparknotes.com/math/algebra1/quadratics/section3/

There can be 0, 1, or 2 solutions to a quadratic equation, depending on whether the expression inside the square root sign, (b [ 2 ]  - 4ac), is positive, negative, or zero. This expression has a special name: the discriminant.
If the discriminant is positive--if b [ 2 ]  -4ac > 0--then the quadratic equation has two solutions.
If the discriminant is zero--if b [ 2 ]  - 4ac = 0--then the quadratic equation has one solution.
If the discriminant is negative--if b [ 2 ]  -4ac < 0--then the quadratic equation has no solutions.

https://www.sparknotes.com/math/algebra1/quadratics/section3/page/2/

https://www.labxchange.org/library/items/lb:LabXchange:10d3270e:html:1

https://statisticsbyjim.com/basics/bimodal-distribution/

In [ ]:
# create 1000 equally spaced points between -1000 and 1000
x = np.linspace(-1000, 1000, 1000)

# calculate the y value for each element of the x vector
y = x ** 2 - 35 * x + 4900  

fig, ax = plt.subplots()
ax.plot(x, y)

In [ ]:
from scipy import stats

In [ ]:
# Probability profit is negative
print(stats.percentileofscore(model_test_results_df['profit'], 0) / 100.0)

***The probability that 
profit will be negative based on this simulation is 22% (the first time I ran this, the histogram was different and calculated 12%)***